In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import datetime
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

from pathlib import Path
import pickle

# Initialize Model

In [7]:
bit_lengths = [7,6,5,4,3,2,1]

In [8]:
model_path = Path('./models')

In [9]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [10]:
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

Files already downloaded and verified
Files already downloaded and verified


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
def create_model(bit_length):
    alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
    
    alexnet.classifier[4] = nn.Linear(4096,1024)
    alexnet.classifier[6] = nn.Linear(1024,10)
    
    alexnet.load_state_dict(torch.load(model_path  / f'dt_quantized_model_bl_{bit_length}',map_location=device))
    alexnet.eval()
    return alexnet

In [15]:
print(device)
for bit_length in bit_lengths:
    alexnet = create_model(bit_length)
    alexnet.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = alexnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Bit Length {bit_length} Accuracy of the network on the 10000 test images: {100 * correct / total} %')

cuda:0


Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
100%|██████████| 2500/2500 [00:40<00:00, 61.72it/s] 
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


Bit Length 7 Accuracy of the network on the 10000 test images: 85.22 %


100%|██████████| 2500/2500 [00:18<00:00, 134.15it/s]
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


Bit Length 6 Accuracy of the network on the 10000 test images: 85.12 %


...

Bit Length 3 Accuracy of the network on the 10000 test images: 83.51 %


100%|██████████| 2500/2500 [00:17<00:00, 145.43it/s]
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


Bit Length 2 Accuracy of the network on the 10000 test images: 83.51 %


100%|██████████| 2500/2500 [00:16<00:00, 147.20it/s]

Bit Length 1 Accuracy of the network on the 10000 test images: 25.24 %


In [17]:
counts_path = Path('./counts')

In [18]:
counts = []
for bit_length in bit_lengths:
    with open(counts_path / f'dt_counts_bl_{bit_length}.pkl', 'rb') as f:
        counts.append(pickle.load(f))
        print(f'{bit_length} bit exponent counts: {counts[-1]}')

7 bit exponent counts: {0: 30518764, 1: 12468009, 2: 1291749, 3: 129113, 4: 12769, 5: 1267, 6: 153, 7: 0}
6 bit exponent counts: {0: 30518764, 1: 12468009, 2: 1291749, 3: 129113, 4: 12769, 5: 1420, 6: 0, 7: 0}
5 bit exponent counts: {0: 30518764, 1: 12468009, 2: 1291749, 3: 129113, 4: 14189, 5: 0, 6: 0, 7: 0}
4 bit exponent counts: {0: 30518764, 1: 12468009, 2: 1291749, 3: 143302, 4: 0, 5: 0, 6: 0, 7: 0}
3 bit exponent counts: {0: 30518764, 1: 12468009, 2: 1435051, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
2 bit exponent counts: {0: 30518764, 1: 13903060, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
1 bit exponent counts: {0: 44421824, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}


In [19]:
alexnetb7 = create_model(7)
alexnetb2 = create_model(2)

Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


In [21]:
alexnetb7.features[0].weight

Parameter containing:
tensor([[[[ 1.1771e-01,  9.3614e-02,  9.4541e-02,  ...,  5.5612e-02,
            2.1318e-02,  4.9124e-02],
          [ 7.4768e-02,  3.8571e-02,  5.2812e-02,  ...,  2.5516e-02,
           -1.1275e-02,  3.7977e-03],
          [ 7.4836e-02,  3.8302e-02,  5.4801e-02,  ...,  4.3016e-02,
            1.0017e-02,  1.2964e-02],
          ...,
          [ 9.1878e-02,  1.0297e-01,  6.6533e-02,  ..., -2.0118e-01,
           -1.2831e-01, -1.1089e-01],
          [ 4.1965e-02,  6.4854e-02,  3.4335e-02,  ..., -2.0219e-01,
           -1.1254e-01, -1.0682e-01],
          [ 4.7127e-02,  6.2411e-02,  2.4200e-02,  ..., -1.1718e-01,
           -9.5527e-02, -8.2790e-02]],

         [[-7.2491e-02, -5.7711e-02, -8.0233e-02,  ..., -5.6304e-04,
           -2.4633e-02,  2.5337e-02],
          [-6.8611e-02, -6.7417e-02, -7.5770e-02,  ..., -3.8183e-03,
           -2.9831e-02,  1.0142e-02],
          [-9.9456e-02, -8.5483e-02, -1.0439e-01,  ..., -2.6303e-02,
           -2.2193e-02,  6.5756e-03]

In [22]:
alexnetb2.features[0].weight

Parameter containing:
tensor([[[[ 0.1177,  0.0936,  0.0945,  ...,  0.0556,  0.0213,  0.0491],
          [ 0.0748,  0.0386,  0.0528,  ...,  0.0255, -0.0113,  0.0000],
          [ 0.0748,  0.0383,  0.0548,  ...,  0.0430,  0.0100,  0.0130],
          ...,
          [ 0.0919,  0.1030,  0.0665,  ..., -0.2012, -0.1283, -0.1109],
          [ 0.0420,  0.0649,  0.0343,  ..., -0.2022, -0.1125, -0.1068],
          [ 0.0471,  0.0624,  0.0242,  ..., -0.1172, -0.0955, -0.0828]],

         [[-0.0725, -0.0577, -0.0802,  ..., -0.0000, -0.0246,  0.0253],
          [-0.0686, -0.0674, -0.0758,  ..., -0.0000, -0.0298,  0.0101],
          [-0.0995, -0.0855, -0.1044,  ..., -0.0263, -0.0222,  0.0000],
          ...,
          [-0.1475, -0.0849, -0.0938,  ...,  0.3083,  0.1788,  0.0804],
          [-0.1430, -0.0736, -0.0715,  ...,  0.2036,  0.1625,  0.0931],
          [-0.0855, -0.0400, -0.0510,  ...,  0.1627,  0.1475,  0.1006]],

         [[-0.0235, -0.0000, -0.0277,  ...,  0.0399, -0.0070,  0.0319],
        

In [23]:
alexnetb2 = create_model(1)

Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


In [24]:
alexnetb2.features[0].weight

Parameter containing:
tensor([[[[ 0.1177,  0.0936,  0.0945,  ...,  0.0556,  0.0213,  0.0491],
          [ 0.0748,  0.0386,  0.0528,  ...,  0.0255, -0.0113,  0.0415],
          [ 0.0748,  0.0383,  0.0548,  ...,  0.0430,  0.0100,  0.0130],
          ...,
          [ 0.0919,  0.1030,  0.0665,  ..., -0.2012, -0.1283, -0.1109],
          [ 0.0420,  0.0649,  0.0343,  ..., -0.2022, -0.1125, -0.1068],
          [ 0.0471,  0.0624,  0.0242,  ..., -0.1172, -0.0955, -0.0828]],

         [[-0.0725, -0.0577, -0.0802,  ..., -0.0598, -0.0246,  0.0253],
          [-0.0686, -0.0674, -0.0758,  ..., -0.0394, -0.0298,  0.0101],
          [-0.0995, -0.0855, -0.1044,  ..., -0.0263, -0.0222,  0.0658],
          ...,
          [-0.1475, -0.0849, -0.0938,  ...,  0.3083,  0.1788,  0.0804],
          [-0.1430, -0.0736, -0.0715,  ...,  0.2036,  0.1625,  0.0931],
          [-0.0855, -0.0400, -0.0510,  ...,  0.1627,  0.1475,  0.1006]],

         [[-0.0235, -0.0211, -0.0277,  ...,  0.0399, -0.0070,  0.0319],
        

In [31]:
np.unique(alexnetb2.features[0].weight.flatten().detach().numpy()).shape

(22628,)

In [32]:
alexnetb4 = create_model(4)
alexnetb3 = create_model(3)
alexnetb1 = create_model(1)
alexnetb2 = create_model(2)
alexnetb5 = create_model(5)
alexnetb6 = create_model(6)


Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


In [33]:
def num_unique_vals(model):
    return np.unique(model.features[0].weight.flatten().detach().numpy()).shape

In [ ]:
np.unique(alexnetb2.features[0].weight.flatten().detach().numpy()).shape

In [34]:
num_unique_vals(alexnetb4)

(20818,)

In [35]:
num_unique_vals(alexnetb3)

(19494,)

In [36]:
num_unique_vals(alexnetb2)

(22628,)

In [37]:
num_unique_vals(alexnetb1)

(22628,)

In [45]:
models = {}
for bit_length in bit_lengths:
    models[bit_length] = create_model(bit_length);

Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.10.0


In [46]:
def num_unique_vals(model):
    return np.unique(model.classifier[4].weight.flatten().detach().numpy()).shape
for bit_length in bit_lengths:
    print(f'Number of unique values for {bit_length} exponent dt quantization: {num_unique_vals(models[bit_length])}')

Number of unique values for 7 exponent dt quantization: (224503,)
Number of unique values for 6 exponent dt quantization: (201257,)
Number of unique values for 5 exponent dt quantization: (187139,)
Number of unique values for 4 exponent dt quantization: (180995,)
Number of unique values for 3 exponent dt quantization: (178947,)
Number of unique values for 2 exponent dt quantization: (178947,)
Number of unique values for 1 exponent dt quantization: (235959,)


In [43]:
alexnet.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=1024, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=1024, out_features=10, bias=True)
)

In [57]:
np.unique(models[1].classifier[4].weight[0].flatten().detach().numpy()).shape

(231,)

In [70]:
l1 = np.load('./layers/4-bit-layers/layer15.npy')
l1

array([[0.04341245],
       [0.04641441],
       [0.05061795],
       ...,
       [0.03959356],
       [0.04256794],
       [0.03307443]], dtype=float32)

In [60]:
l1

array([[0.13047954],
       [0.13544217],
       [0.16758148],
       [0.11127453],
       [0.11510195],
       [0.3035147 ],
       [0.44895065],
       [0.6782765 ],
       [0.534125  ],
       [0.40451404],
       [0.19290188]], dtype=float32)

In [108]:
def estimate_model_size_change(alexnet, bit_length):
    data_type_sizes = []
    abs_max_sizes = []
    count = 0
    bits_in_fp32 = 8 + 23
    for layer in [*alexnet.features,*alexnet.classifier]:
        count += 1 
        # curr_layer_path = curr_path / f'layer{count}.npy' 
        curr_data_size = 0
        curr_abs_max_size = 0
        try:
            data_type_sizes.append(0)
            abs_max_sizes.append(0)
            if len(layer.weight.shape) == 4:
                weights = layer.weight.detach()
                # print(f'Layer {count}')# weights shape pre-quantization: {weights.shape}\nWeights: {weights}')
                for filter in range(0, weights.shape[0]):
                    # print(f'Filter num {filter}')
                    for channel in range(0, weights.shape[1]):
                        # print(f'Channel num {channel}')
                        # print(layer.weight[filter,channel])
                        data_type_sizes[-1] += weights[filter,channel].numel()
                        abs_max_sizes[-1] += weights[filter,channel].shape[0]
                        # for row in range(0,weights.shape[2]):
                        #     weights[filter,channel, row] = quantize_dequantize_dt(weights[filter,channel,row])
                        # print(f'Finish window')
                # print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
                # layer.weight = nn.parameter.Parameter(weights)
                # print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
                # layer.weight = nn.parameter.Parameter(weights)
            else:
                weights = layer.weight.detach()
                # print(f'Layer {count}')# weights shape pre-quantization: {layer.weight.shape}\nWeights: {weights}')
                data_type_sizes[-1] += weights.numel()
                abs_max_sizes[-1] += weights.shape[0]
                print(weights.shape)
                # for row in tqdm(range(0,weights.shape[0])):
                #     weights[row] = quantize_dequantize_dt(weights[row])
                # layer.weight = nn.parameter.Parameter(weights)
                # print(f'Layer {count} weights shape post-quantization: {layer.weight.shape}\nWeights: {weights}')
                # print(layer.weight)
        except (TypeError, AttributeError):
            pass
    return {'data_type_counts': np.array(data_type_sizes), 'data_type_sizes': np.array(data_type_sizes) * bit_length, 'abs_max_counts': np.array(abs_max_sizes), 'abs_max_sizes': np.array(abs_max_sizes) * bits_in_fp32,
           'data_type_sizes_original': np.array(data_type_sizes) * bits_in_fp32}

In [113]:
def bits_to_mb(bits):
    return bits / 8000000

In [125]:
def compute_model_size(model, bit_len):
    bits_in_fp32 = 8 + 23
    results = estimate_model_size_change(model, bit_len)
    data_type_size = results['data_type_sizes'].sum()
    abs_max_size = results['abs_max_sizes'].sum()
    dt_counts = results['data_type_counts'].sum()
    print(f'{data_type_size} bits to represent the {bit_len} quantized dt and {abs_max_size} bits to represent the maxes')
    print(f'{bits_to_mb(data_type_size)} mb to represent the {bit_len} quantized dt and {bits_to_mb(abs_max_size)} mb to represent the maxes')
    print(f'Original Model Size: {bits_to_mb( dt_counts * bits_in_fp32)} mb')
    print(f'This is a {bits_to_mb( dt_counts * bits_in_fp32) / (bits_to_mb(data_type_size)+bits_to_mb(abs_max_size))}x decrease in size')

In [127]:
# estimate_model_size_change(alexnet, 7)

torch.Size([4096, 9216])
torch.Size([1024, 4096])
torch.Size([10, 1024])


{'data_type_counts': array([   23232,        0,        0,   307200,        0,        0,
          663552,        0,   884736,        0,   589824,        0,
               0,        0, 37748736,        0,        0,  4194304,
               0,    10240]),
 'data_type_sizes': array([   162624,         0,         0,   2150400,         0,         0,
          4644864,         0,   6193152,         0,   4128768,         0,
                0,         0, 264241152,         0,         0,  29360128,
                0,     71680]),
 'abs_max_counts': array([  2112,      0,      0,  61440,      0,      0, 221184,      0,
        294912,      0, 196608,      0,      0,      0,   4096,      0,
             0,   1024,      0,     10]),
 'abs_max_sizes': array([  65472,       0,       0, 1904640,       0,       0, 6856704,
              0, 9142272,       0, 6094848,       0,       0,       0,
         126976,       0,       0,   31744,       0,     310]),
 'data_type_sizes_original': array([    720192

In [132]:
compute_model_size(alexnet, 3)

torch.Size([4096, 9216])
torch.Size([1024, 4096])
torch.Size([10, 1024])
133265472 bits to represent the 3 quantized dt and 24222966 bits to represent the maxes
16.658184 mb to represent the 3 quantized dt and 3.02787075 mb to represent the maxes
Original Model Size: 172.134568 mb
This is a 8.743985028285062 x decrease in size
